# Minibatch FFNN Implementation Evaluation
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from MiniBatch import MiniBatch

df = pd.read_csv('dataset.csv')
df

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [2]:
df['outlook'] = pd.Categorical(df['outlook'])
dfOutlook = pd.get_dummies(df['outlook'], prefix = 'outlook')
dfOutlook

,outlook_overcast,outlook_rainy,outlook_sunny
0,0,0,1
1,0,0,1
2,1,0,0
3,0,1,0
4,0,1,0
5,0,1,0
6,1,0,0
7,0,0,1
8,0,0,1
9,0,1,0


In [3]:
df['windy'] = pd.Categorical(df['windy'])
dfWindy = pd.get_dummies(df['windy'], prefix = 'windy')
dfWindy

,windy_False,windy_True
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
5,0,1
6,0,1
7,1,0
8,1,0
9,1,0


In [4]:
df['play'] = df['play'].astype('category')
df['play'] = df['play'].cat.codes

df = pd.concat([df, dfOutlook, dfWindy], axis=1)
df = df.drop(columns=['outlook', 'windy'], axis=1)
df

,temperature,humidity,play,outlook_overcast,outlook_rainy,outlook_sunny,windy_False,windy_True
0,85,85,0,0,0,1,1,0
1,80,90,0,0,0,1,0,1
2,83,86,1,1,0,0,1,0
3,70,96,1,0,1,0,1,0
4,68,80,1,0,1,0,1,0
5,65,70,0,0,1,0,0,1
6,64,65,1,1,0,0,0,1
7,72,95,0,0,0,1,1,0
8,69,70,1,0,0,1,1,0
9,75,80,1,0,1,0,1,0


In [5]:
features = df.drop(columns=['play'], axis=1)
targets = df['play']

print (features.head)
print ()
print (targets.head)

<bound method NDFrame.head of     temperature  humidity  outlook_overcast  outlook_rainy  outlook_sunny  \
0            85        85                 0              0              1   
1            80        90                 0              0              1   
2            83        86                 1              0              0   
3            70        96                 0              1              0   
4            68        80                 0              1              0   
5            65        70                 0              1              0   
6            64        65                 1              0              0   
7            72        95                 0              0              1   
8            69        70                 0              0              1   
9            75        80                 0              1              0   
10           75        70                 0              0              1   
11           72        90                 1   

## Training and Testing

In [6]:
features_train, features_test, targets_train, targets_test = train_test_split(features, targets, test_size=0.5, stratify=targets, random_state=None)
print (features_train)
print ()
print (targets_train)
print ()

print (features_test)
print ()
print (targets_test)
print ()

    temperature  humidity  outlook_overcast  outlook_rainy  outlook_sunny  \
5            65        70                 0              1              0   
11           72        90                 1              0              0   
3            70        96                 0              1              0   
10           75        70                 0              0              1   
13           71        91                 0              1              0   
0            85        85                 0              0              1   
6            64        65                 1              0              0   

    windy_False  windy_True  
5             0           1  
11            0           1  
3             1           0  
10            0           1  
13            0           1  
0             1           0  
6             0           1  

5     0
11    1
3     1
10    1
13    0
0     0
6     1
Name: play, dtype: int8

    temperature  humidity  outlook_overcast  outlook_rainy  o

In [7]:
model = MiniBatch(2, 3, 2, 0.1, 0.5, 4)
model.fit(features_train, targets_train)
predict = model.predict(features_test)

print ('Accuracy\t', accuracy_score(targets_test, predict))

[[2, 1, 0], [3, 5, 4], [6]]
5     0
11    1
3     1
10    1
13    0
0     0
6     1
Name: play, dtype: int8
<class 'pandas.core.series.Series'>


KeyError: 1